# Домашняя работа 6. Бустинг

Максимальная оценка 10 баллов

Ответы на вопросы пишите в комментариях или в markdown ячейках. Таким же образом обозначайте блоки кода для лучшей читаемости (например, "Обучим бэггинг на логистических регрессиях : ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ").

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка: 10**

In [1]:
# !pip install numpy
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

### Задание 1. Градиентный бустинг своими руками  (4 балла)

Вам нужно реализовать упрощенный вариант градиентного бутсинга для задачи регресси. 


**Напоминание, как это работает:**

Обозначим текущую композицию на $N-1$ шаге за $a_{N - 1}(x_i)$. Базовый алгоритм $b_N(x_i)$ обучается на ответах $-\frac{\partial L(y_i, z)}{\partial z}\Bigl|_{z = a_{N - 1}(x_i)}$, где $L(y_i, z)$ — значение функции потерь на объекте при правильном ответе $y_i$ и предсказании $z$. Композиция на следующем шаге получается так:

$$
a_N(x_i) = a_{N-1}(x_i) + \nu\gamma_Nb_N(x_i)
$$

Здесь $\nu \in [0, 1]$ — темп обучения (гиперпараметр), $\gamma_N$ — оптимальный вес, настраиваемый на каждом шаге алгоритма в ходе решения оптимизационной задачи:

$$
\gamma_N = \mathrm{arg}\min_\gamma \frac{1}{\ell}\sum\limits_{i=1}^{\ell}L\left(y_i, a_{N - 1}(x_i) + \gamma b_N(x_i)\right)
$$


Заметьте, что в формуле выше нет $\nu$. Этот гиперпараметр используется для сокращения длины шага, оптимального при составлении композиции $a_N$. Идея отклонения от оптимума должна быть вам уже знакома как способ борьбы с переобучением, когда мы специально форсим модель работать чуть хуже, чем могла бы, на текущем шаге, чтобы сохранить обобщающую способность и не подогнаться под тренировочную выборку (или под шум).

С потерей в 0.5 балла можете принять $\gamma_N = 1$ для каждого $N$. На полный балл необходимо реализовать нахождение оптимального $\gamma_N$ на каждом шаге.

В качестве функции потерь $L$ возьмите MSE.

В качестве базовой модели можете использовать `DecisionTreeRegressor` из `sklearn`.
Для решения оптимизационной задачки можно воспользоваться алгоритмами из любых библиотек, например, `scipy.optimize`, или найти оптимум перебором по сетке из некоторого разумного диапазона.

Можно дописывать свои функции, если необходимо.

In [2]:
class GradientBoosting:
    def __init__(
        self, 
        base_model_class: object = DecisionTreeRegressor,
        base_model_params: dict = {'max_depth': None}, 
        n_estimators: int = 10,
        learning_rate: float = 0.1,
        random_state: int = 0,
        show_progress: bool = True
    ):
        """
        
        Args:
          base_model_class: Class of the base learner.

          base_model_params: Hyperparameters of the base learner.
          
          n_estimators: Number of boosting stages.
          
          learning_rate: Value used to shrink contribution of each base learner to the model. 
          
        """
        
        self.base_model_class = base_model_class
        self.base_model_params = base_model_params
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.random_state = random_state
        self.show_progress = show_progress
        
        # list for optimal gammas at each iteration
        self.gammas = []
        
        # list for base models
        self.models = []

        #last prediction for every object
        self._last_pred = None
        
        
    def find_optimal_gamma(self, 
                           y: np.array, 
                           old_predictions: np.array,
                           new_predictions: np.array) -> float:
        """You may add arguments if it's necessary for your optimization algorithm.
        
        Args:
          y: Target variable.

          old_predictions: Prediction of the additive model at the previous stage.
          
          new_predictions: Prediction of the base learner at the current stage. 
          
        Returns:
          Optimal value for gamma.
          
        """
        return np.dot(y - old_predictions, new_predictions) / np.dot(new_predictions, new_predictions) #vertex of the parabola
    
    
    def _fit_base_model(self, X: np.ndarray, y: np.array):
        """Train one base learner. 
        
        Args:
          X: Feature matrix
          
          y: Target variable.
          
          
        Returns:
          Fitted base learner.
          
        """
        new_model = DecisionTreeRegressor(**self.base_model_params)
        new_model.fit(X, y)
        return new_model


    
        
    def fit(self, X: np.ndarray, y: np.array):
        """Train boosting ("sum" of base learners). 
        
        Args:
          X: Feature matrix
          
          y: Target variable.
          
          
        Returns:
          Fitted boosting.
          
        """
        for _ in tqdm(range(self.n_estimators), disable=(not self.show_progress)):
            
            np.random.seed(seed=self.random_state)
            
            if self._last_pred is None: self._last_pred = np.zeros(y.shape[0])    #setting initial prediction as constant 0

            composition_pred = self._last_pred                  
            residuals = 2 * (composition_pred - y)                                #calculating prediction of the current composition, and the residuals

            model = self._fit_base_model(X, residuals)                            #training base model on residuals
            self.models.append(model)

            base_model_pred = model.predict(X)
            gamma = self.find_optimal_gamma(y, self._last_pred, base_model_pred)  #finding optimal gamma
            self.gammas.append(gamma)
            self._last_pred += self.learning_rate * gamma * base_model_pred       #updating predictions
            

    def predict(self, X: np.ndarray):
        """Make prediction of fitted boosting. 
        
        Args:
          X: Feature matrix


        Returns:
          Prediction of fitted boosting.
          
        """
        y_pred = np.zeros(X.shape[0])
        for i in range(self.n_estimators):
            y_pred += self.learning_rate * self.gammas[i] * self.models[i].predict(X)
        
        return y_pred

*Комментарии:*
1. Добавил поле `_last_pred`, в котором записаны последние предсказания модели по всей выборке
2. Обучаю базовые модели и считаю гамму на bootstrap-подвыборках
3. Добавил параметр random_state в инициализаторе
4. Добавил параметр show_progress, отвечающий за отображение шкалы прогресса в методе `fit`

В остальном все стандартно

Проверьте вашу реализацию на бостонском датасете. Подберите оптимальные гиперпараметры, чтобы победить RandomForestRegressor (не меняйте параметры сида).

In [4]:
housing = fetch_california_housing()
X = housing.data[:5000]
y = housing.target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [5]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_features=4, n_estimators=640, random_state=19052019)

rf.fit(X_train, y_train)
rf_train_error = mean_squared_error(y_train, rf.predict(X_train))
rf_test_error = mean_squared_error(y_test, rf.predict(X_test))
print(rf_train_error, rf_test_error, sep='\n')

0.02283901441436567
0.1673784076585523


Видим, что модель в данном случае переобучена.

In [6]:
"""
careful! runs for around 8 minutes!

for n_estimators in tqdm(range(5, 51, 5)):
    for max_depth in tqdm(range(1, 7)):
        for lr in range(1, 21):
            gb = GradientBoosting(n_estimators=n_estimators, base_model_params={'max_depth' : max_depth}, learning_rate=lr/100, random_state=19052019, show_progress=False)
            gb.fit(X_train, y_train)

            err = mean_squared_error(y_test, gb.predict(X_test))
            if err < rf_error:
                print(f'error: {err}\n n: {n_estimators}\n max_depth: {max_depth}\n learning_rate: {lr / 100}')
"""


"\ncareful! runs for around 8 minutes!\n\nfor n_estimators in tqdm(range(5, 51, 5)):\n    for max_depth in tqdm(range(1, 7)):\n        for lr in range(1, 21):\n            gb = GradientBoosting(n_estimators=n_estimators, base_model_params={'max_depth' : max_depth}, learning_rate=lr/100, random_state=19052019, show_progress=False)\n            gb.fit(X_train, y_train)\n\n            err = mean_squared_error(y_test, gb.predict(X_test))\n            if err < rf_error:\n                print(f'error: {err}\n n: {n_estimators}\n max_depth: {max_depth}\n learning_rate: {lr / 100}')\n"

Изначально вручную удалось засечь, что начиная примерно с `max_depth = 5` и `learning_rate = 5 / n` модель начинает переобучаться, при этом изменения `n` особо не влияют (приходится понижать `learning rate` и получается то же самое, что логично). С помощью перебора выше удалось найти два набора параметров, при которых мы побеждаем случайный лес. При этом видно, что модель сильно переобучается. Мне кажется, что нам просто повезло, шум обучающей и тестовой выборки совпал и получилась относительно низкая ошибка. В реальности логично выбирать другие параметры, при которых на обучающей и тестовой выборке ошибка близка, но все равно какой-то степени переобучения избежать не удастся.

In [7]:
gb = GradientBoosting(n_estimators=40, base_model_params={'max_depth' : 6}, learning_rate=0.17, random_state=19052019, show_progress=True)

gb.fit(X_train, y_train)

print(mean_squared_error(y_train, gb.predict(X_train)),
      mean_squared_error(y_test, gb.predict(X_test)), sep='\n')

100%|██████████| 40/40 [00:01<00:00, 39.71it/s]

0.049874058311388914
0.16659537024916787


### Задание 2. Сравнение подходов (3 балла)

Скачайте данные о выдаче кредитов. Это данные с kaggle, целевая переменная `y` показывает, вернуло ли кредит физическое лицо.

In [8]:
df = pd.read_csv('https://www.dropbox.com/s/uy27mctxo0gbuof/bank_data.csv?dl=1')
df.sample(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
7103,38,admin.,divorced,university.degree,no,yes,no,cellular,aug,fri,...,4,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,-1
2987,33,blue-collar,single,basic.6y,no,yes,no,cellular,may,tue,...,3,999,1,failure,-1.8,92.893,-46.2,1.291,5099.1,1
6480,57,blue-collar,married,unknown,unknown,yes,no,cellular,aug,wed,...,2,999,0,nonexistent,1.4,93.444,-36.1,4.967,5228.1,-1
186,61,admin.,married,university.degree,no,yes,yes,cellular,apr,thu,...,5,999,0,nonexistent,-1.8,93.075,-47.1,1.406,5099.1,1
3299,37,housemaid,married,high.school,no,yes,yes,telephone,may,fri,...,6,999,0,nonexistent,1.1,93.994,-36.4,4.864,5191.0,-1


Решите задачу предсказания возвращения кредита методами, перечисленными ниже:

- Случайный лес
- Бэггинг на деревьях (поставьте для базовых деревьев min_samples_leaf=1)
- Бэггинг, у которого базовой моделью является бустинг с большим числом деревьев (> 100)
- Бэггинг на логистических регрессиях

Используйте логистическую регрессию, случайный лес, `GradientBoostingClassifier` и `BaggingClassifier` из `sklearn`.

1) Какая из моделей имеет лучшее качество? С чем это связано?

2) Какая из моделей сильнее всего переобучается?


Для начала выделим категориальные признаки и применим one-hot encoding.

In [9]:
X = df.loc[:, df.columns != 'y']
y = df.loc[:, "y"]
numeric = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

for column in X.columns[~X.columns.isin(numeric)]:
    one_hot = pd.get_dummies(X[column], prefix=column, dtype=int)
    #print(one_hot)
    X = X.drop(column, axis = 1)
    X = X.join(one_hot)

X.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,39,67,3,999,0,1.4,93.918,-42.7,4.968,5228.1,...,0,0,0,0,1,0,0,0,1,0
1,31,522,1,999,0,1.4,93.918,-42.7,4.960,5228.1,...,0,0,0,1,0,0,0,0,1,0
2,34,84,1,999,0,-1.8,92.893,-46.2,1.250,5099.1,...,0,0,1,0,0,0,0,0,1,0
3,23,332,2,999,0,1.4,93.918,-42.7,4.963,5228.1,...,0,0,0,0,0,0,1,0,1,0
4,63,479,1,999,0,-2.9,92.201,-31.4,0.838,5076.2,...,0,0,0,0,0,1,0,0,1,0


In [10]:
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

rf = RandomForestClassifier(max_features=4, n_estimators=10, bootstrap=False)
dt = BaggingClassifier(estimator=DecisionTreeClassifier(min_samples_leaf=1), n_estimators=10, bootstrap=False)
gb = BaggingClassifier(estimator=GradientBoostingClassifier(n_estimators=200), n_estimators=10, bootstrap=False)
lr = BaggingClassifier(estimator=LogisticRegression(max_iter=5000), n_estimators=10, bootstrap=False)

rf.fit(X_train, y_train)
print('Random Forest trained')
dt.fit(X_train, y_train)
print('Bagging on Desicion Trees trained')
gb.fit(X_train, y_train)
print('Bagging on Gradient Boostings trained')
lr.fit(X_train, y_train)
print('Bagging on Logistic Regressions trained')

rf_train_pred = rf.predict(X_train)
dt_train_pred = dt.predict(X_train)
gb_train_pred = gb.predict(X_train)
lr_train_pred = lr.predict(X_train)

rf_test_pred = rf.predict(X_test)
dt_test_pred = dt.predict(X_test)
gb_test_pred = gb.predict(X_test)
lr_test_pred = lr.predict(X_test)

data = [['Random Forest', 
            mean_squared_error(y_train, rf_train_pred), 
            mean_squared_error(y_test, rf_test_pred), 
            accuracy_score(y_test, rf_test_pred), 
            precision_score(y_test, rf_test_pred), 
            recall_score(y_test, rf_test_pred)],
        ['Bagging on Desicion Trees', 
            mean_squared_error(y_train, dt_train_pred), 
            mean_squared_error(y_test, dt_test_pred), 
            accuracy_score(y_test, dt_test_pred), 
            precision_score(y_test, dt_test_pred), 
            recall_score(y_test, dt_test_pred)],
        ['Bagging on Gradient Boostings', 
            mean_squared_error(y_train, gb_train_pred), 
            mean_squared_error(y_test, gb_test_pred), 
            accuracy_score(y_test, gb_test_pred), 
            precision_score(y_test, gb_test_pred), 
            recall_score(y_test, gb_test_pred)],
        ['Bagging on Logistic Regressions', 
            mean_squared_error(y_train, lr_train_pred), 
            mean_squared_error(y_test, lr_test_pred), 
            accuracy_score(y_test, gb_test_pred), 
            precision_score(y_test, gb_test_pred), 
            recall_score(y_test, gb_test_pred)]]

res = pd.DataFrame(data, columns=['model', 'train error', 'test error', 'accuracy', 'precision', 'recall'])
res

Random Forest trained
Bagging on Desicion Trees trained
Bagging on Gradient Boostings trained
Bagging on Logistic Regressions trained


,model,train error,test error,accuracy,precision,recall
0,Random Forest,0.000000,0.615517,0.846121,0.859634,0.834179
1,Bagging on Desicion Trees,0.000000,0.674138,0.831466,0.841832,0.824027
2,Bagging on Gradient Boostings,0.350575,0.444828,0.888793,0.870192,0.918782
3,Bagging on Logistic Regressions,0.505747,0.563793,0.888793,0.870192,0.918782


### Задание 3. Современные бустинги (3 балла)

Сравните на этих данных любую из трёх популярных имплементаций градиентного бустинга (xgboost, lightgbm, catboost). Подберите основные гиперпараметры (число деревьев, длина шага, глубина дерева/число листьев). Получилось ли круче, чем с моделями выше?

In [11]:
import catboost

ModuleNotFoundError: No module named 'catboost'